<a href="https://colab.research.google.com/github/talhaanwarch/Offensive-Language-Detection/blob/master/OLID_Classification_GloVe_Fastext_biGRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd /content/drive/My Drive/dataset/OLID

/content/drive/My Drive/dataset/OLID


In [3]:
ls

data/          ERNIE/  paragram_300_sl999.txt  wiki-news-300d-1M.vec
elmo-model.h5  glove/  requriemnt.txt          word2vec/


In [4]:
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
import string
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
train=pd.read_csv( 'data/olid-training-v1.0.tsv',sep="\t")
test=pd.read_csv('data/testset-levela.tsv',sep="\t")
y_test=pd.read_csv( 'data/labels-levela.csv',header=None).iloc[:,-1]
#OFF=0 
#NOT=1

In [6]:
y_train=train['subtask_a']
train=train['tweet']
test=test['tweet']
y_train=pd.factorize(y_train)[0]
y_test=pd.factorize(y_test)[0]

import collections
collections.Counter(y_train)

#Counter({0: 4400, 1: 8840})

Counter({0: 4400, 1: 8840})

In [8]:
x=train[323].lower().replace('@user','').replace('url','')
x=re.sub('[^A-Za-z]', ' ', x)
re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','',x)

'   you are grossly misinterpreting who is against this  people across all parties condemn this action  look no further than bill davis and brian mulroney to see that even conservatives think this is a horrible decision '

In [0]:
# Creating the training corpus
stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()
corpus_train = []
for i in train:
    x=i.lower()
    x=x.replace('@user','')
    x=x.replace('@[\w\-]+','')
    #x=x.translate(str.maketrans('', '', string.punctuation))
    x = re.sub('[^A-Za-z]', ' ', x)
    #x=re.sub('\s+',' ',x)
    x=re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','',x) #url
    #x = [lemmatizer.lemmatize(token) for token in x.split(" ")]
    #x = [word for word in x if not word in stop_words]
    #x=" ".join(x)
    corpus_train.append(x)    
# Creating the training corpus
corpus_test = []
for i in test:
    x=i.lower()
    x=x.replace('@user','')
    x=x.replace('@[\w\-]+','')
    #x=x.translate(str.maketrans('', '', string.punctuation))
    x = re.sub('[^A-Za-z]', ' ', x)
    #x=re.sub('\s+',' ',x)
    x=re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','',x) #url
    #x = [lemmatizer.lemmatize(token) for token in x.split(" ")]
    #x = [word for word in x if not word in stop_words]
    #x=" ".join(x)
    corpus_test.append(x) 

In [12]:
corpus_train[0]

' she should ask a few native americans what their take on this is '

In [13]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
max_words = 5000 #frequency of words to be kept
max_len = 200

tokenize = Tokenizer(num_words=max_words)
tokenize.fit_on_texts(corpus_train)
sequences = tokenize.texts_to_sequences(corpus_train)
word_index = tokenize.word_index
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

Using TensorFlow backend.


In [14]:
num_words = min(max_words, len(word_index)) + 1
print(num_words)

5001


In [0]:
embedding_path1 = "wiki-news-300d-1M.vec"
embedding_path2 = "glove/glove.840B.300d.txt"
embed_size = 300

In [0]:
def get_coefs(word,*arr):
    return word, np.asarray(arr, dtype='float32')

def build_matrix(embedding_path, word_index):
    embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path))

    nb_words = min(max_words, len(word_index))
    embedding_matrix = np.zeros((nb_words + 1, embed_size))
    for word, i in word_index.items():
        if i >= max_words:
            continue
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix


In [0]:
fasttext=build_matrix(embedding_path1, word_index)
glove_emb=build_matrix(embedding_path2, word_index)

In [18]:
embedding_matrix=np.mean((fasttext,glove_emb),axis=0)
embedding_matrix.shape

(5001, 300)

In [0]:
test_sequences = tokenize.texts_to_sequences(corpus_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [0]:
from keras import backend as K
def f1(y_true, y_pred):
    '''
    metric from here 
    https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
    '''
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
from keras.callbacks import Callback
class EarlyStoppingByAccuracy(Callback):
    def __init__(self, monitor='val_f1', value=0.87, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current >= self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping THR" % epoch)
            self.model.stop_training = True

In [0]:
callbacks = EarlyStoppingByAccuracy(monitor='val_f1', value=0.90, verbose=1)

In [55]:
from keras.models import Sequential,Model
from keras.layers import Embedding,CuDNNGRU,Dense,Dropout,Bidirectional,GlobalMaxPool1D,GlobalAveragePooling1D, SpatialDropout1D,concatenate,Input
from keras.optimizers import RMSprop
from keras.initializers import Constant
from sklearn.utils import class_weight
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

inp=Input(shape=(max_len, ))
x=Embedding(num_words,embed_size,embeddings_initializer=Constant(embedding_matrix),input_length=max_len,trainable=False)(inp)
x = SpatialDropout1D(0.2)(x)
x=Bidirectional(CuDNNGRU(150, return_sequences = True))(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPool1D()(x)
conc = concatenate([avg_pool, max_pool])


x=Dropout(0.1)(conc)
out=Dense(64, activation="relu")(x)
out=Dense(32, activation="relu")(x)

out=Dense(1, activation="sigmoid")(x)

model=Model(inp,out)

mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')


model.compile(loss='binary_crossentropy', optimizer=RMSprop(0.001), metrics=[f1])

class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
class_weights=dict(enumerate(class_weights))

model.fit(sequences_matrix,y_train,batch_size=256,epochs=10,verbose=2,class_weight=class_weights,
          validation_data=(test_sequences_matrix,y_test),callbacks=[mcp_save,callbacks])

Train on 13240 samples, validate on 860 samples
Epoch 1/10
 - 5s - loss: 0.5959 - f1: 0.7133 - val_loss: 0.4443 - val_f1: 0.8691
Epoch 2/10
 - 4s - loss: 0.5129 - f1: 0.8119 - val_loss: 0.4319 - val_f1: 0.8665
Epoch 3/10
 - 4s - loss: 0.5028 - f1: 0.8185 - val_loss: 0.3979 - val_f1: 0.8840
Epoch 4/10
 - 4s - loss: 0.4925 - f1: 0.8258 - val_loss: 0.4430 - val_f1: 0.8548
Epoch 5/10
 - 4s - loss: 0.4849 - f1: 0.8291 - val_loss: 0.4354 - val_f1: 0.8591
Epoch 6/10
 - 4s - loss: 0.4780 - f1: 0.8317 - val_loss: 0.3960 - val_f1: 0.8761
Epoch 7/10
 - 4s - loss: 0.4720 - f1: 0.8335 - val_loss: 0.4385 - val_f1: 0.8565
Epoch 8/10
 - 4s - loss: 0.4634 - f1: 0.8368 - val_loss: 0.4064 - val_f1: 0.8722
Epoch 9/10
 - 4s - loss: 0.4560 - f1: 0.8401 - val_loss: 0.3920 - val_f1: 0.8762
Epoch 10/10
 - 4s - loss: 0.4499 - f1: 0.8435 - val_loss: 0.3949 - val_f1: 0.8836


In [56]:

print(model.evaluate(test_sequences_matrix,y_test))

860/860 [==============================] - 0s 305us/step
[0.39487432227578273, 0.8824435816254727]


In [57]:
from sklearn.metrics import classification_report

y_pred = model.predict(test_sequences_matrix, batch_size=128, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
y_pred = (y_pred > 0.5)

print(classification_report(y_test, y_pred))


860/860 [==============================] - 0s 250us/step
              precision    recall  f1-score   support

           0       0.70      0.72      0.71       240
           1       0.89      0.88      0.88       620

    accuracy                           0.83       860
   macro avg       0.79      0.80      0.80       860
weighted avg       0.84      0.83      0.83       860



In [58]:
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
f1_score(y_test, y_pred, average="weighted")

0.8344439537443726

In [0]:
import keras
keras.backend.clear_session()